# Introduction


Data are pulled in as large files for ingest. These are external to the
repo folder so as not to bog it down (as repo size is constrained).
From this point there is an *ingest* process and a second process 
described below under the heading **Profiles**. 


To focus on *ingest* for a moment: The idea is to streamline the source
datasets by reducing clutter and cutting out a time window, say one month.
This notebook will start by considering a two year time range from 
1-JAN-2021 through 31-DEC-2022. From there we will focus on shorter
sub-ranges, typically a month in duration or less.



### Inbound data structure



There are two levels of source data time resolution: 1-minute average sampling
and full resolution sampling; which amounts to about 1 sample per second. 
The 1-minute-per-sample data will be smoother via 
averaging with the tradeoff of coarser depth resolution.


Again: Inbound data from OOI is kept outside this repo: In its own `/data/rca` 
folder. This inbound data folder structure is as follows:



```
$HOME/../data/rca/
                  OregonSlopeBase/
                                  platform/                (pc01a)
                                  profiler/                (sf01a)
                                           par/    
                                           do/
                                           vel/
                                           ctd/
                                           fluor/
                                           nitrate/
                                           pco2/
                                           ph/
                                           spkir/
                                           fullres_ctd/ etcetera
                  OregonOffshore/
                                  platform/                (pc01b)
                                  profiler/                (sf01b)
                  AxialBase/ 
                                  platform/               (pc03a)
                                  profiler/               (sf03a)

```

Please note that 1-minute-sample data goes into folders with no prefix; and 
full resolution (1-second-sample) data folders have the `fullres_` prefix 
as in `fullres_ctd`.


There are two resources for shallow profiler data:


- [Ocean Observatories Initiative **Data Explorer**](https://dataexplorer.oceanobservatories.org/)
- [Interactive Oceans **Data Portal**](https://interactiveoceans.washington.edu) which is specific to the Regional Cabled Array




### Terms used


* Regional Cabled Array (RCA): A cabled observatory on the sea floor and in select locations rising up through the water column: off the coast of Oregon
* Site: A location in the RCA
* Platform: A mechanical structure -- static or mobile -- at a site
* Instrument: A device fastened to a platform that carries one or more sensors
* Sensor: A device that measures some aspect of the ocean like pH or temperature
* Stream: Data produced by a sensor as part of an instrument located on a platform at a site in the RCA
* Profiler 'Oregon Slope Base'
* Profiler 'Axial Base' PN3A 
* Profiler 'Oregon Offshore' (shared with the OSU Endurance array)


#### Sensor dictionary with abbreviations


- This table is alphabetized by short abbreviation for the moment


- The instrument name corresponds to the folder structure given above


- Spectral irradiance is abbreviated `spkir` in the OOI nomenclature and this
is retained here. The instrument brings in seven spectral channels (sensors)
with wavelengths in *nm* as indicated.


```
Short     Sensor                Name used     Instrument     Sensor operates
Abbrev.                         here          folder name
-----     ------                -------       ----------     -----------
A         Chlorophyll-A         chlora        fluor          continuous
B         backscatter           backscatter   fluor          continuous
C         CDOM                  cdom          fluor          continuous
D         Density               density       ctd            continuous
E[]       Spec'r optical abs    oa            spec           ? (83 channels)
F[]       Spec'r beam atten     ba            spec
G         pCO2                  pco2          pco2           midnight/noon descent
H         pH                    ph            ph             midnight/noon descent
I         Spectral Irradiance   412nm         spkir          ?
                                443nm
                                490nm
                                510nm
                                555nm
                                620nm
                                683nm
K         Conductivity          conductivity  ctd            continuous
N         Nitrate               nitrate       nitrate        midnight/noon ascent
P         PAR                   par           par            continuous
Q         pressure              pressure      ctd            continuous
O         dissolved oxygen      do            do             continuous
S         salinity              salinity      ctd            continuous
T         temperature           temp          ctd            continuous
U         velocity east         veast         current        continuous:
V         velocity north        vnorth                           from platform
W         velocity up           vup                              looking up
```

### Data download


As noted above: [OOI](https://oceanobservatories.org) 
provides data access via the [Data Explorer](https://dataexplorer.oceanobservatories.org/).
An alternative resource particular to the *OOI cabled array* is the
[Cabled Array data server](https://interactiveoceans.washington.edu).
This site is oriented towards built-in data exploration and is very educator-friendly. 


#### Example download


- [Data access page in the Data Explorer](https://dataexplorer.oceanobservatories.org/#go-to-data-access)
- Access Array Data, Oregon Margin, Profiling Assets, All Instrument Types, All Parameters, Go
    - A very dense page with charts, tabs, multiple scroll bars and so forth
    - Exploration of the various tabs encouraged
        - **Data**, **Inventory**, **Downloads**, **Annotations**, **Deployments**
    - > **Downloads** tab
    - Full resolution data: 1 second per sample
        - Choose link '*full resolution downloads*'
        - Page of fluorometer datasets in deployment sequence: time ranges shown
        - Select Downloads > THREDDS Catalog > Dataset > page listing files
        - Deployment 8, FLORT for the triplet
        - Check the time range in the filename for best match to March 2021
        - Click to go to yet another page, this one for "ACCESS" and multiple choices
            - Select HTTPServer to download the NetCDF data file: 600+ MB
    - 1Min resolution: 1 minute per sample
        - Green **Download** button for various streams / sensors 
        - Three format options: ERDDAP (+ 3 sub-options), CSV and NetCDF
            - NetCDF URL gives a shareable download link
            - NetCDF **Download** button > 1Min downsampled data
                - These datasets extend over the full OOI program duration: 2014 to present
            - Data description text + chart symbol (left) produces a curtain plot


Full resolution data is sub-divided into consecutive time ranges called "deployments". 
The resulting NetCDF files have these distinctions compared to the 1Min data:


- Data is sampled at about one sample per second
- Sensors are combined in NetCDF files
    - For example a CTD file includes salinity, temperature, pressure, density and conductivity
- The data files are much larger on average
- The sample dimension is `obs` (observation number) rather than `row`
    - However the swap procedure is the same: Swap in `time`


#### Code


This notebook defers some functions to the module file `data.py`.

In [1]:
import os, sys, time, glob, warnings
from os.path import join as joindir
from IPython.display import clear_output
from matplotlib import pyplot as plt
from matplotlib import colors as mplcolors
import numpy as np, pandas as pd, xarray as xr
from numpy import datetime64 as dt64, timedelta64 as td64

from data import *

warnings.filterwarnings('ignore')

def doy(theDatetime): return 1 + int((theDatetime - dt64(str(theDatetime)[0:4] + '-01-01')) / td64(1, 'D'))
def dt64_from_doy(year, doy): return dt64(str(year) + '-01-01') + td64(doy-1, 'D')
def day_of_month_to_string(d): return str(d) if d > 9 else '0' + str(d)

print('\nJupyter Notebook running Python {}'.format(sys.version_info[0]))


Jupyter Notebook running Python 3


In [2]:
# operating on '../../../data/rca/OregonSlopeBase/profiler/ctd/ooi-rs01sbps-sf01a-2a-ctdpfa102_6974_57e0_dbda.nc' and so on: July 2021

In [23]:
###### Run this cell to reformat a data file and write it in its new form.
# Example output file: 'axb_nitrate_jan22_nitrate.nc' gives site, instrument, sensor and time range

if not ReformatDataFile(False): print('ReformatDataFile() returned False')



Specify input NetCDF data file

['do', 'vel', 'ctd', 'ph', 'nitrate', 'par', 'fluor', 'pco2', 'fullres_ctd', 'spkir']


Enter index 0, 1, ... to select the instrument:  9


['ooi-rs01sbps-sf01a-3d-spkira101_4efb_11ac_2e12.nc']


Enter index of NetCDF file to use 0







Swap the dimension (e.g. time in place of row or obs)

Dimensions: ['row']
Coordinates: []
Data Variables: ['time', 'spectir_412nm', 'spectir_443nm', 'spectir_490nm', 'spectir_510nm', 'spectir_555nm', 'spectir_620nm', 'spectir_683nm', 'spectir_412nm_qc_agg', 'spectir_443nm_qc_agg', 'spectir_490nm_qc_agg', 'spectir_510nm_qc_agg', 'spectir_555nm_qc_agg', 'spectir_620nm_qc_agg', 'spectir_683nm_qc_agg', 'z']


Rename, Drop or Retain Coordinates



Rename, Drop or Retain Data Variables

For 15 data variables: 0 to drop, non-zero string to rename, enter to retain:

data variable name: spectir_412nm


...

Drop (0), Rename or Keep: 0


data variable name: z


Drop (0), Rename or Keep: 




Drop all Attributes (less any you select)



Ensure the new Dimension is sorted (no User action)



Select output time window (Format yyyy-mm-dd or enter to use the defaults)



Here is the resulting dataset summary view:

<xarray.Dataset>
Dimensions:  (time: 44641)
Coordinates:
  * time     (time) datetime64[ns] 2021-07-01 2021-07-01T00:01:00 ... 2021-08-01
Data variables:
    412nm    (time) float64 0.008115 0.00822 0.007924 ... 0.01103 0.01105
    443nm    (time) float64 0.01464 0.01525 0.0147 ... 0.01601 0.016 0.01552
    490nm    (time) float64 0.01118 0.01134 0.01131 ... 0.01306 0.01247 0.0133
    510nm    (time) float64 0.008455 0.008814 0.008644 ... 0.0112 0.01106
    555nm    (time) float64 -0.003472 -0.004078 ... -0.004942 -0.004373
    620nm    (time) float64 0.006813 0.007422 0.007541 ... 0.008572 0.008338
    683nm    (time) float64 -0.002795 -0.001578 ... -0.000437 -0.0004967
    z        (time) float64 -191.0 -191.0 -191.0 -191.0 ... -193.0 -193.0 -193.0




Enter an output file name. Include the .nc extension (or just enter to skip this):  spkir.nc


In [21]:
ds = xr.open_dataset('chlora.nc')
ds

<xarray.Dataset>
Dimensions:  (time: 43153)
Coordinates:
  * time     (time) datetime64[ns] 2021-07-01 2021-07-01T00:01:00 ... 2021-08-01
Data variables:
    chlora   (time) float64 ...
    z        (time) float64 ...

In [5]:
print(31*24*60)

44640


#### Test dataset

On 10/16/23 the 1min datasets (repo-external /data folder) from OOI were processed for storage
and use in this repository. The file naming convention is 


**`<site>_<instrument>_<month>_<sensor>.nc`**.


Hence for the Oregon Slope Base profiler we have `osb_spkir_jan22_spkir.nc` using the OOI abbreviation
for spectral irradiance **`spkir`**. Note that the sensor and instrument are the same in this case.
In the case of the fluorometer and CTD the sensor is not the same as the instrument.


In this case the time range was constrained Jan 1 to Feb 1 2022. At the moment it is unclear how 
the velocity data is produced: From a sensor on the profiler or from a sensor on the platform?


The only data file containing streams from multiple sensors is the spectral irradiance (spkir) file.
This records 7 wavelength bands. While instruments ctd, fluor and vel have multiple
sensors these are split into separate data files.

In [ ]:
jan22_data = [
'osb_ctd_jan22_pressure.nc',
'osb_ctd_jan22_temperature.nc',
'osb_ctd_jan22_density.nc',
'osb_ctd_jan22_salinity.nc',
'osb_ctd_jan22_conductivity.nc',
'osb_fluor_jan22_fdom.nc',
'osb_fluor_jan22_chlora.nc',
'osb_fluor_jan22_bb.nc',
'osb_spkir_jan22_spkir.nc',
'osb_nitrate_jan22_nitrate.nc',
'osb_pco2_jan22_pco2.nc',
'osb_do_jan22_do.nc',
'osb_par_jan22_par.nc',
'osb_ph_jan22_ph.nc',
'osb_vel_jan22_up.nc',
'osb_vel_jan22_east.nc',
'osb_vel_jan22_north.nc']

#### Data review


The above cell generates a set of about 17 time-bounded NetCDF measurement data files. 
To review these files and their *XArray Dataset data variable* names: Activate and run the
following code. The cell below captures these names into a list of lists.
    
```
data_prefix = '../data/'
for f in jan22_data:
    ds = xr.open_dataset(data_prefix + f)
    print(ds)
    r = input('enter to continue')
```

In [ ]:
# The relative path to the data files from the Notebooks folder is up one level, then to the data folder
data_prefix = '../data/'

# The data variables corresponding to the jan22_data files defined above
jan22_data_vars = [
    ['pressure'],['temperature'],['density'], ['salinity'], ['conductivity'],
    ['fdom'], ['chlora'], ['bb'],
    ['412nm', '443nm', '490nm', '510nm', '555nm', '620nm', '683nm'],
    ['nitrate'],
    ['pco2'],
    ['do'],
    ['par'],
    ['ph'],
    ['up'], ['east'], ['north']]

## Profiles


The final task of this notebook is to produce a table of profile metadata. 


Profile timing is driven by the RCA engineering system, producing some variability in profile
timing. Also shallow profilers occasionally go out of service. Using profile time/depth data we
produce a table of timestamps corresponding to 3-part profile cycles: Ascent, Descent and Rest. 
This table is used in subsequent analysis to isolate individual profiles.


The following cell illustrates a typical day of profiling cycles.


- The time scale for January 10 is in UTM hours
- Nine profiles are run, two of which are noticeably longer in duration
- The two longer profiles at midnight and noon associate with nitrate, pCO2 and pH measurement
    - These have built-in pauses for equilibration during descent
- During the *rest* intervals the profiler is secured in an anchored platform
    - The platform has its own set of instruments

In [ ]:
def ProfilerDepthChart(t0, t1, fnm, annotate=False):
    ds = xr.open_dataset(fnm).sel(time=slice(dt64(t0), dt64(t1)))
    fig, axs = plt.subplots(figsize=(12,4), tight_layout=True)
    axs.plot(ds.time, ds.z, marker='.', ms=11., color='k', mfc='r', linewidth='.0001')
    axs.set(ylim = (-200., 0.), title='Shallow profiler depth with time', ylabel='depth (m)', xlabel='Hours (UTM)')
    if annotate:
        axs.text(dt64('2022-01-09 23:15'), -184, 'AT')
        axs.text(dt64('2022-01-09 23:05'), -193, 'REST')
        axs.text(dt64('2022-01-10 08'), -180, 'midnight')
        axs.text(dt64('2022-01-10 21:40'), -180, 'noon')
        axs.text(dt64('2022-01-10 09:25'), -60, 'slow')
        axs.text(dt64('2022-01-10 09:30'), -70, 'descent')
        axs.text(dt64('2022-01-10 23:00'), -60, 'slow')
        axs.text(dt64('2022-01-10 23:00'), -70, 'descent')
    return True

ProfilerDepthChart('2022-01-10', '2022-01-11', data_prefix + jan22_data[0], True)

print('January 10 2022 profiles')

In [ ]:
fnm = '../data/osb_ctd_jan22_pressure.nc'
a0, a1, d0, d1, r0, r1 = ProfileGenerator(fnm, 'osb', 'z', True)
xr.open_dataset(fnm)['z'][-1440*2:-1].plot()
if not ProfileWriter('../profiles/osb_profiles_jan22.csv', a0, a1, d0, d1, r0, r1): print('ProfileWriter() is False')

#### How to use profile files

- Read the file into a pandas Dataframe
- Each row is a Rest --- Ascent --- Descent phase sequence
    - Each phase has a start and an end: For a total of six events
        - There is degeneracy: r1 == a0, a1 == d0, and mostly d1 == the next row's r0
    - Each entry for these six events is a triple (i, t, z)
        - i is an index from the source file; usually ignore this
        - t is the time of the event; important
        - z is the depth of the event
            - for a1/d0 this will indicate if the profile approached the surface
    - Consequently each row has 18 columns
    
Suppose the idea is to make a depth plot of temperature for a particular profile. Select out this
profile and slice the source data using a time range given as a0 to a1. See the next notebook 
for examples.


In [ ]:
#### visual on one month of profiles

ds = xr.open_dataset("./../data/osb_ctd_jan22_pressure.nc")
fig, axs = plt.subplots(31, 1, figsize=(15,31), tight_layout=True)

for i in range(31):
    daystring = str(i+1) if i > 8 else '0' + str(i+1)
    time0 = dt64('2022-01-' + daystring + 'T00:00:00')
    time1 = dt64('2022-01-' + daystring + 'T23:59:59')
    dsDay = ds.sel(time=slice(time0, time1))
    axs[i].plot(dsDay.time, dsDay.z, marker='.', markersize=1., color='k')
    axs[i].set(ylim = (-200., 0.))

print('...January 2022 Oregon Slope Base daily profiles...')
print('     sub-optimal profiles can be seen on Jan 1, 3, 4, 5 and 7')

#### Comment on profile phase durations


```
  Ascents:   67.4 +-3.0  minutes
  Descents:  47.5 +-24.9          (reason: 2 of 9 each day are longer due to pauses on descent)
  Rests:     44.8 +-14.1
```
